In [1]:
!pip install pandas

In [1]:
import pandas as pd
from gensim.utils import simple_preprocess


In [2]:

df = pd.read_csv('c:/Users/rmessina/Eli/data/song_lyrics_es_1950_2020_cleaned_final.csv', encoding='utf8')
df = df.dropna(subset=['cleaned_lyrics'])

In [ ]:
def preprocess(doc):
    return simple_preprocess(doc, deacc=True)  # Tokenize and remove punctuation


texts = df['cleaned_lyrics'].map(preprocess)

# just 2min

In [4]:
from gensim import corpora

# Create Dictionary
dictionary = corpora.Dictionary(texts)

# Create Corpus: List of bag-of-words vectors for each document
corpus = [dictionary.doc2bow(text) for text in texts]

In [5]:
from gensim.models import LdaModel

num_topics = 32

# Train LDA model
lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,  # Set the number of topics you want
    random_state=42,
    passes=10,
    chunksize=1000
)

lda_model.save(f"c:/Users/rmessina/Eli/models/lda_model_{num_topics}_nohard.gensim")

In [6]:
import pyLDAvis
import pyLDAvis.gensim_models

# Prepare the visualization
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# Display the visualization
pyLDAvis.display(vis)

In [7]:
topics_per_doc = [max(lda_model.get_document_topics(bow), key=lambda x: x[1])[0] for bow in corpus]
df['lda_topic'] = topics_per_doc

pyLDAvis.save_html(vis, 'lda_visualization_nohard.html')
df.to_csv('c:/Users/rmessina/Eli/data/song_lyrics_es_1950_2020_cleaned_nohard_lda.csv', index=False, encoding='utf8')